# Exploring the Hotel-Level Data

Here, we convert all reservations to hotel nightly sales statistics using the `parse_dates` and `res_to_stats` functions in `utils.py`.

We'll start by deriving some basic information about each hotel, including:
* **Capacity** (total number of rooms)
* **Occupancy** (rooms sold / capacity)

Then, we'll pull more statistics into the `stats` DataFrames:
* **Revenue and Rooms Sold by Customer Segment**
* **ADR by Customer Segment**

These stats will help us understand what kind of hotels we're working with.

In [1]:
import pandas as pd
import numpy as np

from utils import parse_dates, add_res_columns, res_to_stats

In [2]:
df_h1 = pd.read_csv("../data/H1.csv")
df_h2 = pd.read_csv("../data/H2.csv")

In [3]:
df_h1 = parse_dates(df_h1)
df_h2 = parse_dates(df_h2)
df_h1 = add_res_columns(df_h1)
df_h2 = add_res_columns(df_h2)

In [4]:
h1_stats = res_to_stats(df_h1)
h2_stats = res_to_stats(df_h2)

KeyboardInterrupt: 

In [5]:
df_h1.columns

Index(['ResNum', 'IsCanceled', 'LeadTime', 'ArrivalDateYear',
       'ArrivalDateMonth', 'ArrivalDateWeekNumber', 'ArrivalDateDayOfMonth',
       'StaysInWeekendNights', 'StaysInWeekNights', 'Adults', 'Children',
       'Babies', 'Meal', 'Country', 'MarketSegment', 'DistributionChannel',
       'IsRepeatedGuest', 'PreviousCancellations',
       'PreviousBookingsNotCanceled', 'ReservedRoomType', 'AssignedRoomType',
       'BookingChanges', 'DepositType', 'Agent', 'Company',
       'DaysInWaitingList', 'CustomerType', 'ADR', 'RequiredCarParkingSpaces',
       'TotalOfSpecialRequests', 'ReservationStatus', 'ReservationStatusDate',
       'ArrivalDate', 'LOS'],
      dtype='object')

In [ ]:
h1_stats.head()

In [ ]:
h2_stats.head(2)

In [ ]:
h1_stats.describe()

In [ ]:
h2_stats.describe()

## Capacity

Based on the above tables, we can see the maximum capacity of each hotel.

**H1 (Resort Hotel)'s capacity is 187 rooms.**

**H2 (City Hotel)'s capacity is 226 rooms.**

In [ ]:
h1_stats["Occ"] = h1_stats.RoomsSold.astype(float) / 187
h2_stats["Occ"] = h2_stats.RoomsSold.astype(float) / 226

In [ ]:
h1_stats.describe()

In [ ]:
h2_stats.describe()

In [6]:
df_h1.head(3)

,ResNum,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,...,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate,ArrivalDate,LOS
0,0,0,342,2015,July,27,1,0,0,2,...,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01,2015-07-01,0
1,1,0,737,2015,July,27,1,0,0,2,...,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01,2015-07-01,0
2,2,0,7,2015,July,27,1,0,1,1,...,NULL,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01,1


In [ ]:
h1_res_nums = np.array(range(len(df_h1)))
h2_res_nums = np.array(range(len(df_h2)))
h1_res_nums, h2_res_nums

In [ ]:
df_h1.CustomerType.value_counts()

In [ ]:
df_h1['Revenue'] = df_h1.LOS * df_h1.ADR
df_h2['Revenue'] = df_h2.LOS * df_h2.ADR

In [ ]:
mask = df_h1.IsCanceled == 0
df_h1[mask][['CustomerType', 'LOS', 'Revenue']].groupby("CustomerType").agg(sum)

In [ ]:
t = (df_h1.groupby("CustomerType")
     .agg({"ADR": "sum", "ResNum": "count"})
     .rename(columns={"ADR":"Revenue", "ResNum": "RoomsSold"}))

In [ ]:
t

In [ ]:
t.loc["Contract", "ADR"]

In [ ]:
import datetime
d1 = datetime.date(2020, 1, 1)
d2 = datetime.date(2020, 4, 1)
all_dates = [datetime.datetime.strftime(d1 + datetime.timedelta(days=x), format="%Y-%m-%d") for x in range((d2-d1).days + 1)]

In [ ]:
dd